# TensorFlow Assignment: Multilayer Perceptron (MLP) Optimizer Sandbox

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: [Weiguo Ye]

## Multilayer Perceptron (MLP)

### Imports and helper functions

Let's play around with some optimizers. First some imports and helper functions:

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Helper functions for creating weight variables
def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Models

And here's the forward pass of the computation graph definition of the completed TensorFlow MLP assignment:

In [3]:
# Model Inputs
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Define the graph
# First fully connected layer
W_fc1 = weight_variable([784, 500])
b_fc1 = bias_variable([500])
# h_fc1 = tf.nn.sigmoid(tf.matmul(x, W_fc1) + b_fc1)
h_fc1 = tf.nn.relu(tf.matmul(x, W_fc1) + b_fc1)

# Second fully connected layer
W_fc2 = weight_variable([500, 10])
b_fc2 = bias_variable([10])
y_mlp = tf.matmul(h_fc1, W_fc2) + b_fc2

# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Optimizers

Instead of the optimizer being given though, let's try out a few. Here we have optimizers implementing algorithms for [Stochastic Gradient Descent](https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer) (SGD), [Stochastic Gradient Descent with Momentum](https://www.tensorflow.org/api_docs/python/tf/train/MomentumOptimizer) (momentum), and [Adaptive Moments](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer) (ADAM). Try out different parameter settings (e.g. learning rate) for each of them.

In [14]:
# Optimizers: Try out a few different parameters for SGD and SGD momentum
train_step_SGD = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cross_entropy)
train_step_momentum = tf.train.MomentumOptimizer(learning_rate=0.001, momentum=0.9).minimize(cross_entropy)
train_step_ADAM = tf.train.AdamOptimizer().minimize(cross_entropy)

# Op for initializing all variables
initialize_all = tf.global_variables_initializer()

### Training

Because we'll be repeating training a few times, let's move our training regimen into function. Note that we pass which optimization algorithm we're running as an argument. In addition to printing out the validation accuracy and final test accuracy, we'll also return the lists of accuracies at each validation step and the training losses at each iteration.

In [15]:
def train_MLP(train_step_optimizer, iterations=4000):
    with tf.Session() as sess:
        # Initialize (or reset) all variables
        sess.run(initialize_all)
        
        # Initialize arrays to track losses and validation accuracies
        valid_accs = [] 
        losses = []
        
        for i in range(iterations):
            # Validate every 250th batch
            if i % 250 == 0:
                validation_accuracy = 0
                for v in range(10):
                    batch = mnist.validation.next_batch(50)
                    validation_accuracy += (1/10.0) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
                print('step %d, validation accuracy %g' % (i, validation_accuracy))
                valid_accs.append(validation_accuracy)
                
            # Train    
            batch = mnist.train.next_batch(50)
            loss, _ = sess.run([cross_entropy, train_step_optimizer], feed_dict={x: batch[0], y_: batch[1]})
            losses.append(loss)
            
        print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
        
    return valid_accs, losses

Finally, let's train the MLP using all three optimizers and compare the results:

In [16]:
print("SGD:")
valid_accs_SGD, losses_SGD = train_MLP(train_step_SGD)
print("Momentum:")
valid_accs_momentum, losses_momentum = train_MLP(train_step_momentum)
print("ADAM:")
valid_accs_ADAM, losses_ADAM = train_MLP(train_step_ADAM)

SGD:
step 0, validation accuracy 0.076
step 250, validation accuracy 0.384
step 500, validation accuracy 0.634
step 750, validation accuracy 0.708
step 1000, validation accuracy 0.752
step 1250, validation accuracy 0.77
step 1500, validation accuracy 0.786
step 1750, validation accuracy 0.806
step 2000, validation accuracy 0.85
step 2250, validation accuracy 0.846
step 2500, validation accuracy 0.842
step 2750, validation accuracy 0.856
step 3000, validation accuracy 0.89
step 3250, validation accuracy 0.848
step 3500, validation accuracy 0.864
step 3750, validation accuracy 0.854
test accuracy 0.8694
Momentum:
step 0, validation accuracy 0.092
step 250, validation accuracy 0.806
step 500, validation accuracy 0.87
step 750, validation accuracy 0.882
step 1000, validation accuracy 0.896
step 1250, validation accuracy 0.93
step 1500, validation accuracy 0.914
step 1750, validation accuracy 0.918
step 2000, validation accuracy 0.91
step 2250, validation accuracy 0.934
step 2500, validatio

### Visualization

Plotting things:

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.tight_layout()

ax[0].plot(valid_accs_SGD)
ax[0].plot(valid_accs_momentum)
ax[0].plot(valid_accs_ADAM)

ax[0].set_ylabel('Validation Accuracy')
ax[0].legend(['SGD', 'Momentum', 'ADAM'], loc='lower right')

ax[1].plot(losses_SGD)
ax[1].plot(losses_momentum)
ax[1].plot(losses_ADAM)

ax[1].set_ylabel('Cross Entropy')
ax[1].legend(['SGD', 'Momentum', 'ADAM'], loc='upper right')
# ax[1].set_ylim([0,1.5])  # <- Use this to change y-axis limits

### Short Answer

How do SGD, SGD with momentum, and ADAM compare in performance? Ease of tuning parameters?

***

As far as accuracy is concerned, SGD is the worse, SGD with momentum is better, and ADAM is the best.
In addition. as for convergence rate, ADAM > SGD with momentum > SGD.

Apparantly, ADAM has more parameters to tune than then other two, and SGD with momentum has one more parameter than the plain SGD. However, the default parameters for ADAM work fine enough, and simply setting momentum as 0.9 is fine enough to guarantee improvement than plain SGD.


With learning rate 0.005, momentum 0.9, and default Adam parameters

SGD:
step 0, validation accuracy 0.108
step 250, validation accuracy 0.762
step 500, validation accuracy 0.826
step 750, validation accuracy 0.844
step 1000, validation accuracy 0.874
step 1250, validation accuracy 0.896
step 1500, validation accuracy 0.888
step 1750, validation accuracy 0.866
step 2000, validation accuracy 0.92
step 2250, validation accuracy 0.906
step 2500, validation accuracy 0.898
step 2750, validation accuracy 0.912
step 3000, validation accuracy 0.914
step 3250, validation accuracy 0.91
step 3500, validation accuracy 0.904
step 3750, validation accuracy 0.902
test accuracy 0.9186
Momentum:
step 0, validation accuracy 0.102
step 250, validation accuracy 0.916
step 500, validation accuracy 0.922
step 750, validation accuracy 0.908
step 1000, validation accuracy 0.954
step 1250, validation accuracy 0.95
step 1500, validation accuracy 0.96
step 1750, validation accuracy 0.946
step 2000, validation accuracy 0.94
step 2250, validation accuracy 0.964
step 2500, validation accuracy 0.944
step 2750, validation accuracy 0.95
step 3000, validation accuracy 0.968
step 3250, validation accuracy 0.962
step 3500, validation accuracy 0.962
step 3750, validation accuracy 0.956
test accuracy 0.9588
ADAM:
step 0, validation accuracy 0.13
step 250, validation accuracy 0.944
step 500, validation accuracy 0.95
step 750, validation accuracy 0.962
step 1000, validation accuracy 0.954
step 1250, validation accuracy 0.97
step 1500, validation accuracy 0.966
step 1750, validation accuracy 0.974
step 2000, validation accuracy 0.988
step 2250, validation accuracy 0.966
step 2500, validation accuracy 0.974
step 2750, validation accuracy 0.974
step 3000, validation accuracy 0.988
step 3250, validation accuracy 0.984
step 3500, validation accuracy 0.98
step 3750, validation accuracy 0.982
test accuracy 0.9704

***

## Convolutional Neural Network (CNN): *Optional*

Adapt the MLP code above to train a CNN instead (*Hint: you can adapt the code from the 01D_MLP_CNN_Assignment_Solutions.ipynb for the CNN just like I did for the MLP*), and again compare the optimizers. The more complex nature of the CNN parameter space means that the differences between optimizers should be much more significant.